**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self, e):
        self.epsilon = e

    def act(self, s, train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
#                 a = np.random.randint(0, self.n_action, size = 1)[0]
                a = np.random.randint(0, self.n_action)
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self, s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Epsilon allows us to switch between exploiting(maximizing rewards in a greedy approach based on the actual knowledge) and exploring(testing new random actions to explore new states)

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python



In [34]:
epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose - reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()

TypeError: unsupported format string passed to NoneType.__format__

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [5]:
class Environment(object):
    def __init__(self, grid_size = 10, max_time = 500, temperature = 0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinates of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale = 16

        self.to_draw = np.zeros((max_time + 2, grid_size * self.scale, grid_size * self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        # b is a list of n=grid_size matrices of size grid_size*3
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        
        #replaces the coefficients of b by 256 if the coefficients of board in same position are positive
        b[self.board > 0,0] = 256
        #replaces the coefficients of b by 256 if the coefficients of board in same position are inferior to 0,2
        b[self.board < 0,2] = 256
        #replace the coefficients of rat position by 256
        b[self.x, self.y, :] = 256
        
        b[-2:,:,:] = 0
        b[:,-2:,:] = 0
        b[:2,:,:] = 0
        b[:,:2,:] = 0
        
        b =  cv2.resize(b, None, fx = self.scale, fy = self.scale, interpolation = cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:] = b
        #return an output ??

    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
                
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
                
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
                
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        #increments time counter
        self.t = self.t + 1
        
        #gets the reward value for the new state (x,y)
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),
                                axis=2)
        state = state[self.x-2 : self.x+3, self.y-2 : self.y+3, :]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        #create a grid size table of bonuses and maluses dependant from the temperature
        
        bonus = 0.5 * np.random.binomial(1, self.temperature, size = self.grid_size**2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size = self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        #replace malus by zero if bonus is positive
        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size, 1),
                               self.position.reshape(self.grid_size, self.grid_size, 1)),
                               axis=2)

        state = state[self.x-2 : self.x+3, self.y-2 : self.y+3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [6]:
# parameters
size = 13

T = 200
temperature = 0.3
epochs_train = 30 # set small when debugging
epochs_test = 10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

position is the position of the rat on the game map and board is the game map

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action)
#         return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
    
    for e in range(epochs):
        ##### FILL IN HERE
        state = env.reset() 
        win = 0
        lose = 0
        game_over = False
        while not game_over :
        
            action = agent.act(state, train=False)
            state, reward, game_over = env.act(action)
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [21]:
# Initialize the game
env = Environment(grid_size = size, max_time = T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix = 'random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.5/15.0. Average score (-4.5)
Win/lose count 13.5/13.0. Average score (-2.0)
Win/lose count 11.0/14.0. Average score (-2.3333333333333335)
Win/lose count 12.0/14.0. Average score (-2.25)
Win/lose count 7.0/11.0. Average score (-2.6)
Win/lose count 10.5/11.0. Average score (-2.25)
Win/lose count 13.0/18.0. Average score (-2.642857142857143)
Win/lose count 9.0/15.0. Average score (-3.0625)
Win/lose count 10.5/14.0. Average score (-3.111111111111111)
Win/lose count 11.0/15.0. Average score (-3.2)
Final score: -3.2


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




## proof1

We have

$$ Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> $$

$$ Q^\pi(s,a)=E_{p^{\pi}}[ r(s_{0},a_{0}) + \sum_{1 \leq t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> $$

$$ Q^\pi(s,a)=E_{p^{\pi}}[ r(s,a) + \gamma \sum_{1 \leq t}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> $$


$$ Q^\pi(s,a)=E_{(s',a')\sim p(.|s,a)}[ r(s,a) + E_{p^{\pi}}[ \gamma \sum_{t}\gamma^{t}r(s_{t}^\prime,a_{t}^\prime)|s_{0}^\prime=s^\prime,a_{0}^\prime=a^\prime] ] \> $$

where $ (s_{t},a_{t}) = (s_{t}^\prime,a_{t}^\prime) $ and $ (s^\prime,a^\prime) = (s_1 , a_1)$  )

Finally

$$ Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] $$

## Proof2

Since:

$$ \pi^*(s) = \max_a Q^*(s,a) $$

then : 

$$ Q^{*}(s,a)=E_{(s',a')\sim \pi^* (.|s,a)}[r(s,a)+\gamma Q^{\pi^*}(s',\pi^*(s'))] $$

then :

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

## Proof3

This following cost function satisfies the Bellman equation:  
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

In fact we saw in the question 2 that the Q-value function is optimal if and only if the difference is null.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory = 2000):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        # we add to the list the move m containing action, state, new state, reward and game_over variable
        if len(self.memory) <= self.max_memory:
            self.memory = self.memory + [m]

    def random_access(self):
        # we sample a sublist from the memory list
#         random_i = np.random.randint(0, len(self.memory), size=1)[0]
        random_i = np.random.randint(0, len(self.memory))
        return self.memory[random_i]

***
The pipeline we will use for training is given below:

In [22]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train = True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix + str(e))

        # Update stats
        score += win - lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights = prefix + 'model.h5', name_model = prefix + 'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [23]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size = 100, batch_size = 16,n_state = 2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        # in order to avoid action values higher than 4 but explain why 
        return np.argmax(self.model.predict(s.reshape([1,
                                                       s.shape[0],
                                                       s.shape[1],
                                                       s.shape[2]])
                                           )[0]
                        )

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        # the rat can see 5² cells
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            # we access a random sample from memory
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()        
            input_states[i] = s_
            
            if game_over_:
                ######## FILL IN
                #since the game is over, there is no additionnal action to perform, thus no future reward to collect
                target_q[i, a_] = r_
                
            else:
                ######## FILL IN
                #we apply the bellman equation to update our target q
                target_q[i, a_] = r_ + self.discount * max(self.model.predict(n_s_.reshape([1,
                                                                                                    s_.shape[0],
                                                                                                    s_.shape[1],
                                                                                                    s_.shape[2]])
                                                                                                    )[0])



        ######## FILL IN
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights = 'model.h5',name_model = 'model.json'):
        self.model.save_weights(name_weights, overwrite = True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights = 'model.h5', name_model = 'model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr = 0.1, **kwargs):
        super(DQN_FC, self).__init__( *args, **kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        #since the model in not able to receive (5,5,2) array as input, we flatten the state to be able train the model
        model.add(Flatten())
        model.add(Dense(10, activation = 'relu', input_shape = (5, 5, self.n_state)))
        model.add(Dense(4, activation = "softmax"))
        model.compile(sgd(lr = lr, decay = 1e-4, momentum = 0.0), "mse")
        self.model = model
        

In [24]:
env = Environment(grid_size = size, max_time = T, temperature = 0.3)
agent = DQN_FC(size, lr = 0.1, epsilon = 0.1, memory_size = 2000, batch_size = 32)
train(agent, env, epochs_train, prefix = 'fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.0216 | Win/lose count 2.0/5.0 (-3.0)
Epoch 001/050 | Loss 0.0126 | Win/lose count 1.5/3.0 (-1.5)
Epoch 002/050 | Loss 0.0335 | Win/lose count 4.0/2.0 (2.0)
Epoch 003/050 | Loss 0.0509 | Win/lose count 4.0/5.0 (-1.0)
Epoch 004/050 | Loss 0.0276 | Win/lose count 2.5/5.0 (-2.5)
Epoch 005/050 | Loss 0.0514 | Win/lose count 5.0/7.0 (-2.0)
Epoch 006/050 | Loss 0.0552 | Win/lose count 4.5/4.0 (0.5)
Epoch 007/050 | Loss 0.0314 | Win/lose count 3.5/4.0 (-0.5)
Epoch 008/050 | Loss 0.0316 | Win/lose count 3.5/6.0 (-2.5)
Epoch 009/050 | Loss 0.0328 | Win/lose count 0.5/6.0 (-5.5)
Epoch 010/050 | Loss 0.0365 | Win/lose count 3.5/4.0 (-0.5)
Epoch 011/050 | Loss 0.0211 | Win/lose count 4.0/3.0 (1.0)
Epoch 012/050 | Loss 0.0168 | Win/lose count 3.0/2.0 (1.0)
Epoch 013/050 | Loss 0.0365 | Win/lose count 2.0/3.0 (-1.0)
Epoch 014/050 | Loss 0.0318 | Win/lose count 3.0/1.0 (2.0)
Epoch 015/050 | Loss 0.0673 | Win/lose count 2.0/1.0 (1.0)
Epoch 016/050 | Loss 0.0235 | Win/lose count 3

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [25]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(64, kernel_size = (2, 2), input_shape = (5, 5, self.n_state), activation = 'relu', padding = 'same'))
        model.add(Conv2D(32, kernel_size = (2, 2), activation = 'relu')) 
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(Adam(lr = lr), "mse")
        self.model = model

In [26]:
epochs_train = 50
env = Environment(grid_size = size, max_time = T, temperature = 0.3)
agent = DQN_CNN(size, lr = .1, epsilon = 0.1, memory_size = 2000, batch_size = 32)
train(agent,env,epochs_train,prefix = 'cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0101 | Win/lose count 2.0/3.0 (-1.0)
Epoch 001/050 | Loss 0.0000 | Win/lose count 5.5/8.0 (-2.5)
Epoch 002/050 | Loss 0.0230 | Win/lose count 3.0/7.0 (-4.0)
Epoch 003/050 | Loss 0.0101 | Win/lose count 13.5/16.0 (-2.5)
Epoch 004/050 | Loss 0.0076 | Win/lose count 13.5/10.0 (3.5)
Epoch 005/050 | Loss 0.0188 | Win/lose count 10.0/15.0 (-5.0)
Epoch 006/050 | Loss 0.0194 | Win/lose count 12.5/9.0 (3.5)
Epoch 007/050 | Loss 0.0213 | Win/lose count 5.0/12.0 (-7.0)
Epoch 008/050 | Loss 0.0061 | Win/lose count 15.5/24.0 (-8.5)
Epoch 009/050 | Loss 0.0078 | Win/lose count 8.0/12.0 (-4.0)
Epoch 010/050 | Loss 0.0214 | Win/lose count 13.5/17.0 (-3.5)
Epoch 011/050 | Loss 0.0245 | Win/lose count 11.5/8.0 (3.5)
Epoch 012/050 | Loss 0.0133 | Win/lose count 17.0/17.0 (0.0)
Epoch 013/050 | Loss 0.0105 | Win/lose count 9.0/20.0 (-11.0)
Epoch 014/050 | Loss 0.0177 | Win/lose count 11.0/10.0 (1.0)
Epoch 015/050 | Loss 0.0200 | Win/lose count 7.5/17.0 (-9.5)
Epoch 016/050 | Loss 0.0

In [27]:
env = Environment(grid_size = size, max_time = T, temperature = 0.7)
agent = DQN_CNN(size, lr = .1, epsilon = 0.1, memory_size = 2000, batch_size = 32)
train(agent,env,epochs_train,prefix = 'cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0367 | Win/lose count 13.0/2.0 (11.0)
Epoch 001/050 | Loss 0.0246 | Win/lose count 10.5/6.0 (4.5)
Epoch 002/050 | Loss 0.0197 | Win/lose count 10.0/2.0 (8.0)
Epoch 003/050 | Loss 0.0090 | Win/lose count 14.5/3.0 (11.5)
Epoch 004/050 | Loss 0.0168 | Win/lose count 5.0/6.0 (-1.0)
Epoch 005/050 | Loss 0.0131 | Win/lose count 2.5/4.0 (-1.5)
Epoch 006/050 | Loss 0.0136 | Win/lose count 5.0/3.0 (2.0)
Epoch 007/050 | Loss 0.0109 | Win/lose count 2.5/1.0 (1.5)
Epoch 008/050 | Loss 0.0226 | Win/lose count 5.5/2.0 (3.5)
Epoch 009/050 | Loss 0.0158 | Win/lose count 7.0/3.0 (4.0)
Epoch 010/050 | Loss 0.0061 | Win/lose count 5.0/8.0 (-3.0)
Epoch 011/050 | Loss 0.0175 | Win/lose count 5.0/4.0 (1.0)
Epoch 012/050 | Loss 0.0023 | Win/lose count 5.5/1.0 (4.5)
Epoch 013/050 | Loss 0.0162 | Win/lose count 3.0/5.0 (-2.0)
Epoch 014/050 | Loss 0.0099 | Win/lose count 5.0/3.0 (2.0)
Epoch 015/050 | Loss 0.0113 | Win/lose count 6.0/4.0 (2.0)
Epoch 016/050 | Loss 0.0004 | Win/lose count 7

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [28]:
env = Environment(grid_size = size, max_time = T, temperature = 0.3)
agent_cnn = DQN_CNN(size, lr = .1, epsilon = 0.1, memory_size = 2000, batch_size = 32)
agent_cnn.load(name_weights = 'cnn_trainmodel.h5', name_model = 'cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr = .1, epsilon = 0.1, memory_size = 2000, batch_size = 32)
agent_cnn.load(name_weights = 'fc_trainmodel.h5', name_model = 'fc_trainmodel.json')

print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix = 'cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix = 'fc_test')

Test of the CNN
Win/lose count 0/0. Average score (0.0)
Win/lose count 1.0/0. Average score (0.5)
Win/lose count 1.5/0. Average score (0.8333333333333334)
Win/lose count 0.5/0. Average score (0.75)
Win/lose count 0.5/0. Average score (0.7)
Win/lose count 0/0. Average score (0.5833333333333334)
Win/lose count 0/0. Average score (0.5)
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 0/0. Average score (0.4444444444444444)
Win/lose count 0/0. Average score (0.4)
Final score: 0.4
Test of the FC
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/2.0. Average score (-1.0)
Win/lose count 0.5/0. Average score (-0.5)
Win/lose count 2.5/0. Average score (0.25)
Win/lose count 1.5/1.0. Average score (0.3)
Win/lose count 1.5/1.0. Average score (0.3333333333333333)
Win/lose count 0.5/0. Average score (0.35714285714285715)
Win/lose count 0.5/3.0. Average score (0.0)
Win/lose count 1.5/6.0. Average score (-0.5)
Win/lose count 0.5/0. Average score (-0.4)
Final score: -0.4


In [29]:
HTML(display_videos('cnn_test9.mp4'))

In [30]:
HTML(display_videos('fc_test9.mp4'))

**Impact of temperature increase**

With a higher value of temperature, we notice higher rewards because of the increased probability of presence of cheese in comparison with poison

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [37]:
def train_explore(agent, env, epoch, prefix = ''):
    # Number of won games
    score = 0
    loss = 0
    agent.set_epsilon(0.8)
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        
        
        while not game_over:
            # The agent performs an action
            agent.set_epsilon(agent.epsilon * 0.8)
            action = agent.act(state, train = True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix + str(e))

        # Update stats
        score += win - lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights = prefix + 'model.h5', name_model = prefix + 'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size = 10, max_time = 500, temperature = 0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        # we add malus_position as a attribute of this new environment class
        self.malus_position = np.zeros((grid_size, grid_size))

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinates of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale = 16

        self.to_draw = np.zeros((max_time + 2, grid_size * self.scale, grid_size * self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        # b is a list of n=grid_size matrices of size grid_size*3
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        
        #replaces the coefficients of b by 256 if the coefficients of board in same position are positive
        b[self.board > 0,0] = 256
        #replaces the coefficients of b by 256 if the coefficients of board in same position are inferior to 0,2
        b[self.board < 0,2] = 256
        #replace the coefficients of rat position by 256
        b[self.x, self.y, :] = 256
        
        b[-2:,:,:] = 0
        b[:,-2:,:] = 0
        b[:2,:,:] = 0
        b[:,:2,:] = 0
        
        b =  cv2.resize(b, None, fx = self.scale, fy = self.scale, interpolation = cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:] = b
        #return an output ??

    def act(self, action, train = True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
                
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
                
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
                
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        #increments time counter
        self.t = self.t + 1
        
        #gets the reward value for the new state (x,y)
        reward = 0
        if train:
            reward = - self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
 
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((
                                self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),
                                axis = 2)
        state = state[self.x-2 : self.x+3, self.y-2 : self.y+3, :]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        #create a grid size table of bonuses and maluses dependant from the temperature
        
        bonus = 0.5 * np.random.binomial(1, self.temperature, size = self.grid_size**2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size = self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time + 2, self.grid_size * self.scale, self.grid_size * self.scale, 3))

        #replace malus by zero if bonus is positive
        malus[bonus>0] = 0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        #initialize malus_position to a table of zeros
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        state = np.concatenate((
                               self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size, 1),
                               self.position.reshape(self.grid_size, self.grid_size, 1)),
                               axis=2)

        state = state[self.x-2 : self.x+3, self.y-2 : self.y+3, :]
        return state

In [38]:
# Training
epochs_train = 50
env = EnvironmentExploring(grid_size = size, max_time = T, temperature = 0.3)
agent = DQN_CNN(size, lr = .1, epsilon = 0.8, memory_size = 2000, batch_size = 32,n_state = 3)
train_explore(agent, env, epochs_train, prefix = 'cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0474 | Win/lose count 3.5/27.800000000000068 (-24.300000000000068)
Epoch 001/050 | Loss 0.0100 | Win/lose count 4.5/22.50000000000006 (-18.00000000000006)
Epoch 002/050 | Loss 0.0324 | Win/lose count 7.0/32.200000000000124 (-25.200000000000124)
Epoch 003/050 | Loss 0.0219 | Win/lose count 7.5/26.100000000000044 (-18.600000000000044)
Epoch 004/050 | Loss 0.0372 | Win/lose count 9.0/25.100000000000037 (-16.100000000000037)
Epoch 005/050 | Loss 0.0129 | Win/lose count 11.5/28.20000000000011 (-16.70000000000011)
Epoch 006/050 | Loss 0.0365 | Win/lose count 11.5/27.80000000000008 (-16.30000000000008)
Epoch 007/050 | Loss 0.0185 | Win/lose count 11.5/27.20000000000006 (-15.70000000000006)
Epoch 008/050 | Loss 0.0275 | Win/lose count 11.0/29.800000000000065 (-18.800000000000065)
Epoch 009/050 | Loss 0.0371 | Win/lose count 9.5/31.30000000000011 (-21.80000000000011)
Epoch 010/050 | Loss 0.0237 | Win/lose count 10.5/30.100000000000065 (-19.600000000000065)
Epoch 011/050 |

In [39]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore9.mp4'))

Win/lose count 1.5/20.600000000000026. Average score (-19.100000000000026)
Win/lose count 0/19.70000000000001. Average score (-19.40000000000002)
Win/lose count 0.5/19.900000000000013. Average score (-19.400000000000016)
Win/lose count 1.0/24.100000000000076. Average score (-20.32500000000003)
Win/lose count 1.5/21.400000000000034. Average score (-20.24000000000003)
Win/lose count 0.5/22.200000000000045. Average score (-20.483333333333366)
Win/lose count 0.5/21.600000000000037. Average score (-20.571428571428605)
Win/lose count 0.5/19.80000000000001. Average score (-20.41250000000003)
Win/lose count 0.5/23.10000000000006. Average score (-20.655555555555587)
Win/lose count 1.0/22.000000000000043. Average score (-20.690000000000033)
Final score: -20.690000000000033


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***